In [283]:
import numpy as np
import scipy as sp

from scipy.stats import norm

In [284]:
S0 = 100
T = 0.5
K = 90
r = 0.05
q = 0.02
sigma = 0.5
n_sim = 0
n_rep = 0
n = 100

## Black-Scholes

$$ c(S_0, 0) = S_0N(d_1) - Ke^{-rT}N(d_2)$$
$$d1 = \frac{\ln(\frac{S_0}{K}) + (r-q+\frac{\sigma^2}{2})T}{\sigma\sqrt{T}} \quad\quad d2 = d1 - \sigma\sqrt{T}$$

In [285]:
d1 = np.log(S0 / K) + ( r - q + (sigma**2) / 2) * T
d1 = d1 / (sigma* np.sqrt(T))
d2 = d1 - (sigma* np.sqrt(T))

print("-->", np.exp(-q*T)* S0 * norm.cdf(d1) - K * np.exp(-r*T)*norm.cdf(d2))

--> 19.461057884772558


## Monte Carlo

In [286]:

avg = []
tmp = []
for i in range(20):
    payoff = np.random.normal(np.log(S0) + (r - q - (sigma**2) / 2) * T, (sigma) * np.sqrt(T), 10000)
    payoff = np.exp(payoff)
    for num in payoff:
        if num >= K:
            tmp.append(num - K)
        else:
            tmp.append(0)
    avg.append(np.exp(-r * T) * np.mean(tmp))
    print(avg[-1])

19.10003254167035
19.258138131846103
19.35350338024557
19.35913272681825
19.42252448126177
19.353444789930432
19.38499448521067
19.45144614855389
19.46165016135612
19.43725684579184
19.465051512790144
19.489684821414752
19.490032075381347
19.480133311565073
19.504953332044558
19.511729487003063
19.496066529342592
19.47764424118561
19.50650028680449
19.468290543149916


In [287]:
np.mean(avg) - 2 * np.std(avg), np.mean(avg) + 2 * np.std(avg)

(19.22608184057399, 19.621139142762665)

## CRR Binomial Tree

In [288]:
delta_t = T/n
u = np.exp(sigma * np.sqrt(delta_t))
d = 1 / u
p = (np.exp((r-q)*delta_t) - d)/(u-d)
print("delta t:{}\nu:{}\t d:{}\np:{}".format(delta_t, u, d, p))

delta t:0.005
u:1.0359877703222138	 d:0.965262359891545
p:0.49328312337215086


In [289]:
crr_tree = np.zeros((n+1, n+1))
crr_tree.shape

(101, 101)

### European option

In [290]:
cnj = []

In [291]:
for col in range(n, -1, -1): # n~0
    print(col, end =' ')
    if col == n:
        for row in range(0, n+1): # 0~n
    #         print(row, col)
            stock_value = S0 * (u ** (n-row)) * (d ** row)
            crr_tree[row, col] =max(0, stock_value - K)
    else:
#         print(row, col)
        for row in range(0, col+1):
#             print(crr_tree[row, col], crr_tree[row, col + 1], crr_tree[row + 1, col + 1])
            crr_tree[row, col] = np.exp(-r*delta_t)* (p * crr_tree[row, col + 1] + (1 - p) * crr_tree[row + 1, col + 1])

100 99 98 97 96 95 94 93 92 91 90 89 88 87 86 85 84 83 82 81 80 79 78 77 76 75 74 73 72 71 70 69 68 67 66 65 64 63 62 61 60 59 58 57 56 55 54 53 52 51 50 49 48 47 46 45 44 43 42 41 40 39 38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 0 

In [292]:
crr_tree[0,0]

19.490566482943247

### American option

In [293]:
cnj = []

In [294]:
crr_tree = np.zeros((n+1, n+1))
crr_tree.shape

(101, 101)

In [295]:
for col in range(n, -1, -1): # n~0
    print(col, end =' ')
    if col == n:
        for row in range(0, n+1): # 0~n
    #         print(row, col)
            stock_value = S0 * (u ** (n-row)) * (d ** row)
            crr_tree[row, col] =max(0, stock_value - K)
    else:
#         print(row, col)
        for row in range(0, col+1):
            stock_value = S0 * (u ** (col-row)) * (d ** row)
            crr_tree[row, col] = np.exp(-r*delta_t)* (p * crr_tree[row, col + 1] + (1 - p) * crr_tree[row + 1, col + 1])
            crr_tree[row, col] = max(crr_tree[row, col], stock_value - K)

100 99 98 97 96 95 94 93 92 91 90 89 88 87 86 85 84 83 82 81 80 79 78 77 76 75 74 73 72 71 70 69 68 67 66 65 64 63 62 61 60 59 58 57 56 55 54 53 52 51 50 49 48 47 46 45 44 43 42 41 40 39 38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 0 

In [296]:
crr_tree[0,0]

19.490868043292895

## Bonus

### European

In [297]:
cnj = []

In [298]:
for col in range(n, -1, -1): # n~0
    print(col, end =' ')
    if col == n:
        for row in range(0, n+1): # 0~n
    #         print(row, col)
            stock_value = S0 * (u ** (n-row)) * (d ** row)
            cnj.append(max(0, stock_value - K))
    else:
#         print(row, col)
        for row in range(0, col+1):
#             print(crr_tree[row, col], crr_tree[row, col + 1], crr_tree[row + 1, col + 1])
            cnj[row] = np.exp(-r*delta_t)* (p * cnj[row] + (1 - p) * cnj[row + 1])

100 99 98 97 96 95 94 93 92 91 90 89 88 87 86 85 84 83 82 81 80 79 78 77 76 75 74 73 72 71 70 69 68 67 66 65 64 63 62 61 60 59 58 57 56 55 54 53 52 51 50 49 48 47 46 45 44 43 42 41 40 39 38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 0 

In [299]:
cnj[0]

19.490566482943247

### American

In [300]:
cnj = []

In [301]:
for col in range(n, -1, -1): # n~0
    print(col, end =' ')
    if col == n:
        for row in range(0, n+1): # 0~n
    #         print(row, col)
            stock_value = S0 * (u ** (n-row)) * (d ** row)
            cnj.append(max(0, stock_value - K))
    else:
#         print(row, col)
        for row in range(0, col+1):
            stock_value = S0 * (u ** (col-row)) * (d ** row)
            cnj[row] = np.exp(-r*delta_t)* (p * cnj[row] + (1 - p) * cnj[row + 1])
            cnj[row] = max(cnj[row], stock_value - K)

100 99 98 97 96 95 94 93 92 91 90 89 88 87 86 85 84 83 82 81 80 79 78 77 76 75 74 73 72 71 70 69 68 67 66 65 64 63 62 61 60 59 58 57 56 55 54 53 52 51 50 49 48 47 46 45 44 43 42 41 40 39 38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 0 

In [302]:
cnj[0]

19.490868043292895

In [71]:
def large_fac(number):
    largest_num = 100000
    result = [0] * largest_num
    result[0] = 1
    for i in range(1, number+1):
        prod = 0
        for j in range(len(result)):
            prod = result[j] * i + prod
            result[j] = prod % 10
            prod = prod // 10
    result.reverse()
    for i in range(largest_num):
        if result[i] != 0:
            for j in range(i, largest_num):
                print(result[j], end='')
            break

In [73]:
large_fac(1000)

4023872600770937735437024339230039857193748642107146325437999104299385123986290205920442084869694048004799886101971960586316668729948085589013238296699445909974245040870737599188236277271887325197795059509952761208749754624970436014182780946464962910563938874378864873371191810458257836478499770124766328898359557354325131853239584630755574091142624174743493475534286465766116677973966688202912073791438537195882498081268678383745597317461360853795345242215865932019280908782973084313928444032812315586110369768013573042161687476096758713483120254785893207671691324484262361314125087802080002616831510273418279777047846358681701643650241536913982812648102130927612448963599287051149649754199093422215668325720808213331861168115536158365469840467089756029009505376164758477284218896796462449451607653534081989013854424879849599533191017233555566021394503997362807501378376153071277619268490343526252000158885351473316117021039681759215109077880193931781141945452572238655414610628921879602238389714760